In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier
import os
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')
# from sklearn.model_selection import train_test_split
import itertools
from sklearn import preprocessing
from tensorflow.keras.optimizers import Adam
from sklearn.impute import SimpleImputer



In [2]:
youtubeCSV = pd.read_csv('./../data/youtube_data/youtube_finalmerge.csv')

In [3]:
viemoCSV = pd.read_csv('./../data/viemo_data/viemo_final.csv')

In [4]:
youtubeCSV['Label'] = 'YouTube'

In [5]:
viemoCSV['Label'] = 'viemo'

In [6]:
# newdf = youtubeCSV.add(viemoCSV)
pcap_data = pd.concat([youtubeCSV, viemoCSV], axis=0)

In [7]:
pcap_data

,No.,Time,Source,Destination,Protocol,Length,Info,Label
0,1,0.000000,Syrotech_8e:b1:98,Broadcast,0xfffa,60,Ethernet II,YouTube
1,2,0.042970,192.168.1.35,172.217.160.238,TCP,1514,61871 > 443 [ACK] Seq=1 Ack=1 Win=8196 Len=1...,YouTube
2,3,0.042974,192.168.1.35,172.217.160.238,TCP,1514,61871 > 443 [ACK] Seq=1461 Ack=1 Win=8196 Le...,YouTube
3,4,0.042974,192.168.1.35,172.217.160.238,TLSv1.2,686,Application Data,YouTube
4,5,0.042975,192.168.1.35,172.217.160.238,TCP,1514,[TCP Out-Of-Order] 61871 > 443 [ACK] Seq=1 A...,YouTube
...,...,...,...,...,...,...,...,...
1721847,1721848,72468.453355,192.168.152.133,162.247.243.30,TCP,54,"43944 > 443 [FIN, ACK] Seq=91183 Ack=3378 Wi...",viemo
1721848,1721849,72468.453431,162.247.243.30,192.168.152.133,TCP,60,443 > 43944 [ACK] Seq=3378 Ack=91184 Win=642...,viemo
1721849,1721850,72468.455805,192.168.152.1,224.0.0.22,IGMPv3,60,Membership Report / Leave group 224.168.100.1,viemo
1721850,1721851,72468.502061,192.168.152.1,224.0.0.22,IGMPv3,60,Membership Report / Join group 224.168.100.1 f...,viemo


In [8]:
viemoCSV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1721852 entries, 0 to 1721851
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   No.          int64  
 1   Time         float64
 2   Source       object 
 3   Destination  object 
 4   Protocol     object 
 5   Length       int64  
 6   Info         object 
 7   Label        object 
dtypes: float64(1), int64(2), object(5)
memory usage: 105.1+ MB


In [9]:
pcap_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2454807 entries, 0 to 1721851
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   No.          int64  
 1   Time         float64
 2   Source       object 
 3   Destination  object 
 4   Protocol     object 
 5   Length       int64  
 6   Info         object 
 7   Label        object 
dtypes: float64(1), int64(2), object(5)
memory usage: 168.6+ MB


In [10]:
pcap_data.reset_index()

,index,No.,Time,Source,Destination,Protocol,Length,Info,Label
0,0,1,0.000000,Syrotech_8e:b1:98,Broadcast,0xfffa,60,Ethernet II,YouTube
1,1,2,0.042970,192.168.1.35,172.217.160.238,TCP,1514,61871 > 443 [ACK] Seq=1 Ack=1 Win=8196 Len=1...,YouTube
2,2,3,0.042974,192.168.1.35,172.217.160.238,TCP,1514,61871 > 443 [ACK] Seq=1461 Ack=1 Win=8196 Le...,YouTube
3,3,4,0.042974,192.168.1.35,172.217.160.238,TLSv1.2,686,Application Data,YouTube
4,4,5,0.042975,192.168.1.35,172.217.160.238,TCP,1514,[TCP Out-Of-Order] 61871 > 443 [ACK] Seq=1 A...,YouTube
...,...,...,...,...,...,...,...,...,...
2454802,1721847,1721848,72468.453355,192.168.152.133,162.247.243.30,TCP,54,"43944 > 443 [FIN, ACK] Seq=91183 Ack=3378 Wi...",viemo
2454803,1721848,1721849,72468.453431,162.247.243.30,192.168.152.133,TCP,60,443 > 43944 [ACK] Seq=3378 Ack=91184 Win=642...,viemo
2454804,1721849,1721850,72468.455805,192.168.152.1,224.0.0.22,IGMPv3,60,Membership Report / Leave group 224.168.100.1,viemo
2454805,1721850,1721851,72468.502061,192.168.152.1,224.0.0.22,IGMPv3,60,Membership Report / Join group 224.168.100.1 f...,viemo


In [11]:
pcap_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2454807 entries, 0 to 1721851
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   No.          int64  
 1   Time         float64
 2   Source       object 
 3   Destination  object 
 4   Protocol     object 
 5   Length       int64  
 6   Info         object 
 7   Label        object 
dtypes: float64(1), int64(2), object(5)
memory usage: 168.6+ MB


In [12]:
pcap_data

,No.,Time,Source,Destination,Protocol,Length,Info,Label
0,1,0.000000,Syrotech_8e:b1:98,Broadcast,0xfffa,60,Ethernet II,YouTube
1,2,0.042970,192.168.1.35,172.217.160.238,TCP,1514,61871 > 443 [ACK] Seq=1 Ack=1 Win=8196 Len=1...,YouTube
2,3,0.042974,192.168.1.35,172.217.160.238,TCP,1514,61871 > 443 [ACK] Seq=1461 Ack=1 Win=8196 Le...,YouTube
3,4,0.042974,192.168.1.35,172.217.160.238,TLSv1.2,686,Application Data,YouTube
4,5,0.042975,192.168.1.35,172.217.160.238,TCP,1514,[TCP Out-Of-Order] 61871 > 443 [ACK] Seq=1 A...,YouTube
...,...,...,...,...,...,...,...,...
1721847,1721848,72468.453355,192.168.152.133,162.247.243.30,TCP,54,"43944 > 443 [FIN, ACK] Seq=91183 Ack=3378 Wi...",viemo
1721848,1721849,72468.453431,162.247.243.30,192.168.152.133,TCP,60,443 > 43944 [ACK] Seq=3378 Ack=91184 Win=642...,viemo
1721849,1721850,72468.455805,192.168.152.1,224.0.0.22,IGMPv3,60,Membership Report / Leave group 224.168.100.1,viemo
1721850,1721851,72468.502061,192.168.152.1,224.0.0.22,IGMPv3,60,Membership Report / Join group 224.168.100.1 f...,viemo


In [13]:

def getBurstSeries(pcap_data): 
    # pcap_data = pd.read_csv('./../../youtube_data/youtube_finalmerge.csv')
    #pcap_data = pd.read_csv(filePath)
    print(pcap_data.shape)
    pcap_data.head()
    print(pcap_data.info())
    # Filter TCP flows
    tcp_flows = pcap_data[pcap_data['Protocol'] == 'TCP']


    # Group TCP flows by source and destination
    grouped_flows = tcp_flows.groupby(['Source', 'Destination'])

        # Function to calculate bytes per second (BPS), packets per second (PPS), and average packet length (PLEN)
    def calculate_features(flow):
        time_diff = flow['Time'].diff().fillna(0)  # Time difference between packets
        byte_diff = flow['Length'].diff().fillna(0)  # Byte difference between packets
        
        # Bytes per second (BPS)

        flow['BPS'] = byte_diff / time_diff 
        # Replace NaN and inf with zeros in BPS
        flow['BPS'].replace([np.nan, np.inf], 0, inplace=True)
        
        # Packets per second (PPS)
        flow['PPS'] = 1 / time_diff 

        # Replace NaN and inf with zeros in PPS
        flow['PPS'].replace([np.nan, np.inf], 0, inplace=True)
        
        # Average packet length (PLEN)
        flow['PLEN'] = byte_diff / flow['PPS']
        flow['PLEN'].replace([np.nan, np.inf], 0, inplace=True)        
        

        flow['Label'] = flow['Label'].apply(lambda x: 1 if x == 'YouTube' else 2)

        
        return flow

    processed_flows = grouped_flows.apply(calculate_features)
    processed_flows = processed_flows.drop(columns=['No.', 'Source', 'Destination', 'Protocol', 'Info'])
    processed_flows.set_index('Time', inplace=True)
    # Group by 0.25-second intervals and calculate the mean
    aggregated_features = processed_flows.groupby(np.ceil(processed_flows.index / .250)).mean(numeric_only=True).fillna(0)
    
    # Threshold for burst detection
    # I = 0.5  # I is in seconds
    I = 2

    # Initialize variables to store burst series
    burst_series = []

    # Initialize variables to keep track of burst
    burst_start_index = None
    burst_sum = 0

    # Iterate through the time series data
    for i in range(1, len(aggregated_features)):
        # Calculate time difference between consecutive points
        # time_diff = (aggregated_features.index[i] - aggregated_features.index[i-1]) / 1000  # Convert to seconds
        time_diff = (aggregated_features.index[i] - aggregated_features.index[i-1])  # Convert to seconds
        
        # Check if time difference is less than threshold
        if time_diff < I:
            # If burst has not started yet, mark the start index
            if burst_start_index is None:
                burst_start_index = i - 1
            
            # Add the value of the point to burst sum
            burst_sum += aggregated_features.iloc[i]['Length']  # the value of each point in the time series
        
        else:
            # If burst was ongoing, add the burst sum to burst series
            if burst_start_index is not None:
                burst_series.append(burst_sum)
                # Reset burst variables
                burst_start_index = None
                burst_sum = 0

    # If burst was ongoing at the end of the time series, add the burst sum to burst series
    if burst_start_index is not None:
        burst_series.append(burst_sum)


    # Create a custom index based on 250-millisecond intervals
    custom_index = pd.timedelta_range(start=0, periods=len(burst_series), freq='250ms')
    # Create a Series with burst_series data and custom index
    burst_series_with_index = pd.Series(burst_series, index=custom_index)
    # Resample the Series into 250-millisecond intervals
    aggregated_bursts = burst_series_with_index.resample('250ms').sum().fillna(0)
    # print(aggregated_bursts)

    # return aggregated_bursts
    return aggregated_features

# aggFeatures = getBurstSeries(y1)

In [14]:
aggFeatures = getBurstSeries(pcap_data)

(2454807, 8)
<class 'pandas.core.frame.DataFrame'>
Index: 2454807 entries, 0 to 1721851
Data columns (total 8 columns):
 #   Column       Dtype  
---  ------       -----  
 0   No.          int64  
 1   Time         float64
 2   Source       object 
 3   Destination  object 
 4   Protocol     object 
 5   Length       int64  
 6   Info         object 
 7   Label        object 
dtypes: float64(1), int64(2), object(5)
memory usage: 168.6+ MB
None


In [15]:
aggFeatures['Label'] = aggFeatures['Label'].astype(int)

In [17]:
aggFeatures=aggFeatures.reset_index()

In [19]:
aggFeatures = aggFeatures[np.isfinite(aggFeatures).all(1)]

In [20]:
aggFeatures

,Time,Length,Label,BPS,PPS,PLEN
0,0.0,93.000000,2,0.000000e+00,0.000000,0.000000
1,1.0,420.888889,1,-4.599714e+07,264104.087276,-3.275684
2,2.0,57.000000,2,0.000000e+00,0.000000,0.000000
3,5.0,91.444444,1,1.300977e+06,130836.575961,0.093238
4,6.0,60.000000,1,-2.160059e+03,100009.774024,-5.652738
...,...,...,...,...,...,...
12709,491957.0,66.000000,1,-1.968717e+03,55836.615966,-20.317800
12710,491959.0,66.000000,1,0.000000e+00,0.171498,0.000000
12711,491976.0,61.000000,1,1.638493e+01,83337.950469,-10.036342
12712,491979.0,66.000000,1,0.000000e+00,0.177783,0.000000


In [22]:
np.all(np.isfinite(aggFeatures))

True

In [23]:
# aggFeatures = aggFeatures[np.isfinite(aggFeatures).all(1)]


In [26]:
aggFeatures

,Time,Length,Label,BPS,PPS,PLEN
0,0.0,93.000000,2,0.000000e+00,0.000000,0.000000
1,1.0,420.888889,1,-4.599714e+07,264104.087276,-3.275684
2,2.0,57.000000,2,0.000000e+00,0.000000,0.000000
3,5.0,91.444444,1,1.300977e+06,130836.575961,0.093238
4,6.0,60.000000,1,-2.160059e+03,100009.774024,-5.652738
...,...,...,...,...,...,...
12709,491957.0,66.000000,1,-1.968717e+03,55836.615966,-20.317800
12710,491959.0,66.000000,1,0.000000e+00,0.171498,0.000000
12711,491976.0,61.000000,1,1.638493e+01,83337.950469,-10.036342
12712,491979.0,66.000000,1,0.000000e+00,0.177783,0.000000


In [27]:
df = aggFeatures

In [28]:
total_rows = df.shape[0]
print(" total numaber of rows " ,total_rows)
# Select ratio
ratio = 0.80
train_size = int(total_rows*ratio)

# Split data into test and train
train = df[0:train_size]
# Remove column name 'A'
#train = train.drop(columns=['Time'])
total_rows = train.shape[0]
print(" total numaber of rows in training" ,total_rows)
test = df[train_size:]
total_rows = test.shape[0]
print(" total numaber of rows in testing  " ,total_rows)

 total numaber of rows  11667
 total numaber of rows in training 9333
 total numaber of rows in testing   2334


In [29]:
train

,Time,Length,Label,BPS,PPS,PLEN
0,0.0,93.000000,2,0.000000e+00,0.000000,0.000000
1,1.0,420.888889,1,-4.599714e+07,264104.087276,-3.275684
2,2.0,57.000000,2,0.000000e+00,0.000000,0.000000
3,5.0,91.444444,1,1.300977e+06,130836.575961,0.093238
4,6.0,60.000000,1,-2.160059e+03,100009.774024,-5.652738
...,...,...,...,...,...,...
10369,417566.0,66.000000,1,0.000000e+00,229.237894,0.000000
10370,417568.0,61.000000,1,1.091939e+01,41671.333092,-9.980981
10371,417570.0,241.000000,1,-1.995410e+03,36936.196156,188.705475
10372,417572.0,62.000000,1,0.000000e+00,83333.306971,0.000000


In [30]:
test

,Time,Length,Label,BPS,PPS,PLEN
10374,417574.0,66.0,1,-1638.766664,56137.498262,-24.408600
10375,417576.0,66.0,1,0.000000,2902.171079,0.000000
10376,417587.0,66.0,1,0.000000,20736.846427,0.000000
10377,417595.0,83.0,1,2.161588,0.093982,244.727498
10378,417596.0,60.0,1,0.000000,0.090969,0.000000
...,...,...,...,...,...,...
12709,491957.0,66.0,1,-1968.717086,55836.615966,-20.317800
12710,491959.0,66.0,1,0.000000,0.171498,0.000000
12711,491976.0,61.0,1,16.384930,83337.950469,-10.036342
12712,491979.0,66.0,1,0.000000,0.177783,0.000000


In [32]:
# set index column
train.set_index('Time', inplace=True)
test.set_index('Time', inplace=True)

In [37]:
# select feature and target variable
X = train.loc[:,train.columns != 'Label']
y = train.loc[:,train.columns == 'Label']
X.shape, y.shape

((9333, 4), (9333, 1))

In [39]:
# convert to numpy
X= X.to_numpy()
y= y.to_numpy()
 
y= y.flatten()

In [44]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def getAccuracyMetrics(confusionMatrix ):
    print(" >>>> ")
    print(confusionMatrix)
    conf_matrix = np.array(confusionMatrix)

    # Calculate true positives, false positives, true negatives, false negatives
    TP = conf_matrix[1, 1]
    FP = conf_matrix[0, 1]
    TN = conf_matrix[0, 0]
    FN = conf_matrix[1, 0]

    # Calculate accuracy
    accuracy = (TP + TN) / np.sum(conf_matrix)

    # Calculate precision
    precision = TP / (TP + FP)

    # Calculate recall
    recall = TP / (TP + FN)

    # Calculate F1 score
    f1 = 2 * (precision * recall) / (precision + recall)

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

In [45]:
# define and train the Tabnet model with cross validation
kf = KFold(n_splits=5, random_state=42, shuffle=True)
CV_score_array    =[]
for train_index, test_index in kf.split(X):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    tb_cls = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-3),
                       scheduler_params={"step_size":10, "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax' # "sparsemax"
                       )
    print("================================================================")
    print(X_train)
    print(y_train)
    print("====")
    print(X_valid)
    print(y_valid)
    print("==== > ")
    tb_cls.fit(X_train,y_train,
               eval_set=[(X_train, y_train), (X_valid, y_valid)],
               eval_name=['train', 'valid'],
               eval_metric=['accuracy'],
               max_epochs=1000 , patience=100,
               batch_size=28, drop_last=False)            
    CV_score_array.append(tb_cls.best_cost)

    
    pred=tb_cls.predict(X_train)
    print("================================================================")
    # print(classification_report(y_valid, pred))
    conf_matrix = confusion_matrix(y_train, pred)
    print(f"Confusion Matrix: \n {conf_matrix}\n")

    getAccuracyMetrics(conf_matrix)

[[ 4.20888889e+02 -4.59971360e+07  2.64104087e+05 -3.27568356e+00]
 [ 5.70000000e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.00000000e+01 -2.16005943e+03  1.00009774e+05 -5.65273800e+00]
 ...
 [ 6.10000000e+01  1.09193889e+01  4.16713331e+04 -9.98098100e+00]
 [ 2.41000000e+02 -1.99541002e+03  3.69361962e+04  1.88705475e+02]
 [ 1.46600000e+03  3.03557916e+02  2.16827083e-01  6.45675800e+03]]
[1 2 1 ... 1 1 1]
====
[[ 9.30000000e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 9.14444444e+01  1.30097676e+06  1.30836576e+05  9.32376667e-02]
 [ 8.07213115e+02 -2.30886495e+07  1.30718527e+05  6.03628285e+01]
 ...
 [ 6.60000000e+01  0.00000000e+00  9.76586773e-02  0.00000000e+00]
 [ 2.41000000e+02 -2.13937525e+03  1.35879508e+05  1.41431903e+03]
 [ 6.20000000e+01  0.00000000e+00  8.33333070e+04  0.00000000e+00]]
[2 1 1 ... 1 1 1]
==== > 
epoch 0  | loss: 0.46297 | train_accuracy: 0.75636 | valid_accuracy: 0.77343 |  0:00:01s
epoch 1  | loss: 0.35353 | train_accuracy: 0.7